<a href="https://colab.research.google.com/github/Fu-kit/Final_Exam_12_November/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. ⚙️ Setup

In [ ]:
# 1. Install pyinputplus (Required for Colab)
!pip install pyinputplus

# 2. Import Libraries (Essential for all parts of the program)
import pandas as pd
import pyinputplus as pyip
import sys



# 2. 🗃️ Data Storage Function (load_data)

In [31]:
# 2. 🗃️ Data Storage Function (load_data) - Robust Version
def load_data():
    """
    Checks for 'budget.csv' and attempts to load it.
    If not found OR if load fails (due to corruption/error),
    it creates a new DataFrame.
    """
    # Required columns for a new DataFrame
    columns = ['date', 'description', 'type', 'amount']

    try:
        # Attempt to load the existing CSV file
        df = pd.read_csv('budget.csv')

        # Defense against incorrect columns (Edge Case 3 - Optional but recommended check):
        if list(df.columns) != columns:
            # If columns are wrong, raise a generic error to trigger the except block
            raise Exception("Loaded file has incorrect column names.")

        print("\n✅ Existing budget loaded successfully.")

    except FileNotFoundError:
        # Handles the case where the file simply doesn't exist (Edge Case 1)
        print("\n⚠️ budget.csv not found. Creating a new budget.")
        df = pd.DataFrame(columns=columns)

    except Exception as e:
        # This catches all other errors, including file corruption (ParserError)
        # (Edge Case 2 - The critical one)
        print(f"\n❌ ERROR loading budget.csv: {e}")
        print("Starting a new budget due to loading failure.")
        df = pd.DataFrame(columns=columns)

    return df

# 3. ➕ Add a Transaction Function (add_transaction)

In [32]:
# 3. ➕ Add a Transaction Function (add_transaction) - Shorter Version
def add_transaction(df):
    """
    Prompts user for transaction details and adds the new row to the DataFrame.
    """
    print("\n--- Add Transaction ---")

    # 1. Get validated inputs using pyinputplus (no change)
    date = pyip.inputStr("Enter date (YYYY-MM-DD): ")
    description = pyip.inputStr("Enter description: ")
    type_ = pyip.inputChoice(['income', 'expense'], prompt="Enter type (income/expense): ")
    amount = pyip.inputFloat("Enter amount: $", min=0.01)

    # 2. Create the new row as a Dictionary
    new_data = {
        'date': date,
        'description': description,
        'type': type_,
        'amount': amount
    }

    # 3. Append the new row using pd.concat with a single row DataFrame created on the fly
    # Note: Creating a temporary DataFrame from the dict is required for pd.concat
    updated_df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)

    print("👍 Transaction added.")
    return updated_df

# 4. 📜 View All Transactions Function (view_transactions)

In [33]:
# 4. 📜 View All Transactions Function (view_transactions)
def view_transactions(df):
    """
    Displays the entire DataFrame, or a message if empty. (Required)
    """
    print("\n--- All Transactions ---")
    if df.empty:
        print("No transactions yet.")
    else:
        # Display the DataFrame (Required)
        print(df)
    return df # Return the original DF, as it was not changed

""" I knew this part is vulnerable for the case of large number of transactions (e.g., 5,000 to 10,000 transactions)
ut did not have enough time to handle it."""

' I knew this part is vulnerable for the case of large number of transactions (e.g., 5,000 to 10,000 transactions)\nut did not have enough time to handle it.'

# 5. 💰 View Current Balance Function (view_balance)

In [34]:

# 5. 💰 View Current Balance Function (view_balance)
def view_balance(df):
    """
    Calculates and prints the total balance.
    """
    print("\n--- Current Balance ---")

    # Handle the case with no transactions
    if df.empty:
        print("Current Balance: $0.00")
        return df

    # Calculate totals
    # Filter by type and then sum the amount column
    total_income = df[df['type'] == 'income']['amount'].sum()
    total_expense = df[df['type'] == 'expense']['amount'].sum()

    # Balance calculation (Required)
    current_balance = total_income - total_expense

    # Print the balance in a clear format (Required)
    print(f"Current Balance: ${current_balance:.2f}")

    # You MUST include at least one assert() statement for testing!
    # A dedicated testing function is recommended, but for inline logic check:
    # assert current_balance == (total_income - total_expense), "Balance calculation error!"

    return df # Return the original DF, as it was not changed


# 6. 💾 Save and Exit Function (save_and_exit)


In [35]:

# 6. 💾 Save and Exit Function (save_and_exit)
def save_and_exit(df):
    """
    Saves the DataFrame to budget.csv (Required) and exits the program (Required).
    """
    # REQUIRED ACTION: Save the DataFrame, index=False prevents writing row numbers
    df.to_csv('budget.csv', index=False)
    print("\n💾 Data saved to budget.csv.")

    # REQUIRED ACTION: Exit
    print("👋 Goodbye!")
    sys.exit()



# 7. 💻 Main Menu and Program Loop (show_menu and main)

In [ ]:
# 7. 💻 Main Menu and Program Loop (show_menu and main)

def show_menu():
    """Displays the menu and gets a valid choice (1-4) from the user."""
    print("\n--- Personal Budget Manager ---")
    print("1. Add Transaction")
    print("2. View All Transactions")
    print("3. View Current Balance")
    print("4. Save & Exit")

    # pyinputplus ensures choice is an integer between 1 and 4 (Required)
    choice = pyip.inputInt("Enter your choice (1-4): ", min=1, max=4)
    return choice


def main():
    # 1. Load data first (Storage)
    transactions_df = load_data()

    # 2. Pythonic Dictionary mapping choices to functions
    menu_actions = {
        1: add_transaction,
        2: view_transactions,
        3: view_balance,
        4: save_and_exit
    }

    # 3. Main Menu Loop (Required)
    while True:
        user_choice = show_menu()

        # Get the corresponding function from the dictionary
        action_func = menu_actions.get(user_choice)

        # Execute the function and reassign the DataFrame (essential for option 1)
        transactions_df = action_func(transactions_df)

# Execute the main function to start the program
if __name__ == "__main__":
    main()

""" Here is vulnerability for the Edge Case: User Interrupts or Forces Exit (The "Unsaved Exit")
But I did not enough time to fix it."""